In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from scipy import sparse
from sys import getsizeof
import numpy as np
import os
import datetime as dt
import meteomatics.api as metapi

from __future__ import print_function

directory = r"D:\Users\xubil\OneDrive\Documents\Wildfires Data NPZ"
data = {}
processed = {}

# Get xgboost model

In [24]:

username = 'student_xu_bill'
password = '7o7zT54NlW'

In [2]:
def change_zoom(x_min, x_max, y_min, y_max):
    ratio_y, ratio_x = (y_max-y_min)/36400, (x_max-x_min)/59700
    

    for subdir in data:
        processed[subdir] = {}

        for filename in data[subdir]:

            data_process = data[subdir][filename][y_min:y_max, x_min:x_max] # To csc speeds up processing

            N, M = data_process.shape
            s, t = int(400*ratio_y), int(400*ratio_x)          # decimation factors for y and x directions
            T = sparse.csc_matrix((np.ones((M,)), np.arange(M), np.r_[np.arange(0, M, t), M]), (M, (M-1) // t + 1))
            S = sparse.csr_matrix((np.ones((N,)), np.arange(N), np.r_[np.arange(0, N, s), N]), ((N-1) // s + 1, N))
            result = S @ data_process @ T     # downsample by binning into s x t rectangles
            processed[subdir][filename] = result.todense() 
    
    return processed

In [5]:
for filename in os.listdir(directory):
    print("Reading from ",filename)
    filedir = os.path.join(directory, filename)
    if (filename == "fires" or filename == "Ignore"):
        continue

    data[filename] = {}

    for npzname in os.listdir(filedir):
        # print(npzname)
        finaldir = os.path.join(filedir, npzname)
        data[filename][npzname] = sparse.load_npz(finaldir).tocsc()

Reading from  aq_routes_2020_l_arc_f
Reading from  bdat_hypso_l_arc
Reading from  canards_sauv_mhb_2008_s_poly
Reading from  canards_sauv_mhnb_2008_s_poly
Reading from  data-npz_bdtq_hydro_s_poly
Reading from  dmti_transmissionlines_2021_l_arc
Reading from  fires
Reading from  Ignore
Reading from  mrnf_20k_peu_ecofo_ori_poly_f
